# Model

## Constants

In [26]:
BATCH_SIZE = 32
EPOCHS=10

## Preparing the data

In [3]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [4]:
# Data Transforms
transform_train = transforms.Compose([transforms.Resize((224,224)),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
                                      transforms.ColorJitter(brightness=1, contrast=1, saturation=1),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

transform_test = transforms.Compose([transforms.Resize((224,224)),
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [6]:
# Reading the dataset from the folders
training_data = datasets.ImageFolder('datasets/processed/train', transform=transform_train)
validation_data = datasets.ImageFolder('datasets/processed/validation', transform=transform_test)
testing_data = datasets.ImageFolder('datasets/processed/test', transform=transform_test)

In [9]:
# Initializing the data loaders
training = DataLoader(training_data, batch_size=BATCH_SIZE, shuffle=True)
validation = DataLoader(validation_data, batch_size=BATCH_SIZE, shuffle=True)
testing = DataLoader(testing_data, batch_size=BATCH_SIZE, shuffle=False)

## The Model

In [15]:
from torchvision import models
import torch.nn as nn
import torch

In [20]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Running the model in ', device.type)

Running the model in  cpu


In [21]:
model = models.mobilenet_v2(pretrained=True)

In [22]:
for param in model.features.parameters():
    param.requires_grad = False

In [23]:
n_inputs = model.classifier[1].in_features
last_layer = nn.Linear(n_inputs, 32)
model.classifier[1] = last_layer
model.to(device)

MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

## Training the model

In [25]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
losses = []
accuracy = []

In [ ]:
for i in range(0, EPOCHS):
    epoch_loss = 0.0
    epoch_correct = 0

    for inputs, labels in training:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        
        epoch_loss += loss.item()
        epoch_correct += torch.sum(preds == labels.data).cpu().numpy()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    average_loss = epoch_loss/len(training.dataset)
    epoch_accuracy = (epoch_correct/len(training.dataset))*100
    
    losses.append(average_loss)
    accuracy.append(epoch_accuracy)

    print("EPOCH: {} Loss: {} Accuracy: {}".format(i+1, average_loss, epoch_accuracy))

In [27]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(range(0, len(losses)), losses, label="Training Loss")

In [ ]:
plt.plot(range(0, len(accuracy)), accuracy, label="Training Accuracy")